In [9]:
import pandas as pd
import numpy
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import  classification_report
import numpy as np
import chime

In [2]:
# Define the neural network model
class Classifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

    def train(self, train_loader, criterion, optimizer, num_epochs):
        for epoch in range(num_epochs):
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
    
    def predict(self, inputs):
        with torch.no_grad():
            outputs = self(inputs)
            _, predicted = torch.max(outputs, 1)
        return predicted

In [3]:
# Load atmospheric CRN metric data

abiotic_flux=pd.read_csv('Archean Earth flux network metrics, no life.csv') # abiotic case
biotic_flux=pd.read_csv('Archean Earth flux network metrics, with life.csv') # biotic case
abiotic_steady_state=pd.read_csv('Archean Earth steady state network metrics, no life.csv') # weird abiotic case

exo_combined=pd.concat([abiotic_flux,biotic_flux,abiotic_steady_state])
exo_data=exo_combined[['Mean degree','Average shortest path length','CH4 abundance']]
exo_target=exo_combined['Has life?']

In [4]:
# Define classifier input values
input_size = exo_data.shape[1]
hidden_size = 100
num_classes=2 # because, hey, a planet either has life, or it doesn't!
learning_rate = 0.01
batch_size = 32
num_epochs = 100

X_train, X_test, y_train, y_test = train_test_split(exo_data,exo_target, test_size=0.2, random_state=23) 


In [5]:
# Convert the data into PyTorch tensors, create datasets, and then tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) 

In [10]:
# Train Model

model = Classifier(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
model.train(train_loader, criterion, optimizer, num_epochs)
chime.success()


In [11]:
# Test model

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_pred = model.predict(X_test_tensor)
y_pred_np = np.array(y_pred)

correct = sum(y_test == y_pred_np)
accuracy = correct / len(y_test)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred_np))

Accuracy: 0.8533419857235561
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      1123
           1       0.98      0.47      0.63       418

    accuracy                           0.85      1541
   macro avg       0.91      0.73      0.77      1541
weighted avg       0.87      0.85      0.83      1541



In [ ]:
Not too 